<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/RNN_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
def download_and_read(url):
  local_file = url.split('/')[-1]
  local_file = local_file.replace("%20", " ")
  p = tf.keras.utils.get_file(local_file, url, extract=True, cache_dir=".")
  local_folder = os.path.join("drive/MyDrive/Datasets/UCI_ML_ES", local_file.split('.')[0])
  labeled_sentences = []
  for labeled_filename in os.listdir(local_folder):
    if labeled_filename.endswith("_labelled.txt"):
      with open(os.path.join(local_folder, labeled_filename), "r") as f:
        for line in f:
          sentence, label = line.strip().split("\t")
          labeled_sentences.append((sentence, label))

  return labeled_sentences
labeled_sentences = download_and_read(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"
)
sentences = [s for (s, l) in labeled_sentences]
labels = [l for (s, l) in labeled_sentences]